In [ ]:
from pathlib import Path
import os
import shutil
from datasets import load_dataset, load_metric
import pandas as pd

BASE_DIR = Path().cwd().parent
wikiann_dl_dir = BASE_DIR.joinpath("data/interim")
task = 'ner'

In [ ]:
!pip3 install datasets transformers seqeval --user

In [ ]:
!pip3 install cufflinks plotly chart-studio tqdm==4.43.0

In [ ]:
# Standard plotly imports
from chart_studio import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

## Подгружаем данные

In [ ]:
shutil.rmtree(str(wikiann_dl_dir.joinpath('cache')))


os.mkdir(str(wikiann_dl_dir.joinpath('cache')))

datasets = load_dataset(str(wikiann_dl_dir.joinpath("wikiann_local.py")), data_files=str(wikiann_dl_dir.joinpath("default.zip")), 
                        cache_dir=str(wikiann_dl_dir.joinpath('cache')))

datasets

In [ ]:
df = pd.DataFrame(datasets['train'])

Получаем словарь тегов для дальнейшего преобразования датафрейма к виду: токен-тег-номер предложения

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list
label_list = dict(enumerate(label_list))

### Анализ в разрезе токенов

Преобразуем Датафрейм к виду токен-тег-номер предложения

In [ ]:
res = []
for i, v in enumerate(df[['tokens', 'ner_tags']].values):
    res.extend(zip([f'sent_{i}']*len(v[0]), v[0],[label_list[i] for i in v[1]]))
df_tegs = pd.DataFrame(res, columns=['SENTENCE', 'TOKEN', 'TEG'])

In [ ]:
df_tegs.info()

Часточное распрпделеине тегов

In [ ]:
df_tegs['TEG'].iplot(kind='hist', xTitle='TEG',
                  yTitle='count', title='TEG Distribution')

Добавляем столбец Сущность - BIO-разметку приводим просто к сущностям

In [ ]:
df_tegs['ENT'] = df_tegs['TEG'].apply(lambda x: x.split('-')[-1])

Частотное распределение сущностей

In [ ]:
df_tegs['ENT'].iplot(kind='hist', xTitle='ENT',
                  yTitle='count', title='ENT Distribution')

### Анализ в разрезе спанов

In [ ]:
# df['spans'].values[0][0].split(':')

def mske_span_teg(sent, dfs):
    rr = []
    for s in dfs:
        split = s.split(':')
        rr.append([sent, split[0], ''.join(split[1:]).strip()])
    return rr



res = []
for i, v in enumerate(df[['spans']].values[:]):
    sent = f'sent_{i}'
    vv = mske_span_teg(sent, v[0])
#     print(vv)
    
    res.extend(vv)
df_spans = pd.DataFrame(res, columns=['SENTENCE','TEG', 'SPAN'])

In [ ]:
df_spans.info()

In [ ]:
df_spans['TEG'].iplot(kind='hist', xTitle='TEG',
                  yTitle='count', title='TEG Distribution')

In [ ]:
df_spans[df_spans['TEG'] == 'PER']

In [ ]:
df_spans[['SENTENCE', 'TEG']].pivot(columns='SENTENCE', values='TEG').iplot(
        kind='box',
        yTitle='TEG',
        title='TEG Distribution by SENTENCE')